In [32]:
tf.keras.backend.clear_session()

In [5]:
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers.convolutional import Conv3D, MaxPooling3D
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import GlobalMaxPool2D
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import Reshape, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.utils import np_utils, generic_utils
from tensorflow.keras.callbacks import ModelCheckpoint

import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn import preprocessing

In [6]:
nb_classes = 83

In [7]:

print('Building model: ', end="")
model = Sequential()
#`channels_last` corresponds to inputs with shape `(batch, spatial_dim1, spatial_dim2, spatial_dim3, channels)`
strides = (1,1,1)
kernel_size = (3, 3, 3)
model.add(Conv3D(32, kernel_size, strides=strides, activation='relu', padding='same', input_shape=(16, 64, 96, 3)))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)

model.add(Conv3D(64, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)

model.add(Conv3D(128, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(1,8,12)))
print(model.output_shape)

model.add(Reshape((16, 256)))
print(model.output_shape)
model.add(LSTM(256, return_sequences=True))
print(model.output_shape)
model.add(LSTM(256))
print(model.output_shape)

model.add(Dense(256, activation='relu'))
print(model.output_shape)

model.add(Dense(nb_classes, activation='softmax'))
print(model.output_shape)
print('done')

Building model: (None, 16, 64, 96, 32)
(None, 16, 32, 48, 32)
(None, 16, 32, 48, 64)
(None, 16, 16, 24, 64)
(None, 16, 16, 24, 128)
(None, 16, 8, 12, 128)
(None, 16, 8, 12, 256)
(None, 16, 8, 12, 256)
(None, 16, 8, 12, 256)
(None, 16, 1, 1, 256)
(None, 16, 256)
(None, 16, 256)
(None, 256)
(None, 256)
(None, 83)
done


In [8]:
classes = [' Scroll_hand_towards_right',
 'Scroll_hand_towards_left',
 'Scroll_hand_downward',
 'Scroll_hand_upward',
'Scroll_hand_forward',
 'Scroll_hand_backward',
 'Cross_index_fingers',
 'Zoom_in_with_fists',
 'Zoom_out_with_fists',
 'Rotate_fists_clockwise',
 'Rotate_fists_counterclockwise',
 'Zoom_in_with_fingers',
 'Zoom_out_with_fingers',
 'Rotate_fingers_clockwise',
 'Rotate_fingers_counterclockwise',
 'Click_with_index_finger',
 'Sweep_diagonal',
 'Sweep_circle',
 'Sweep_cross',
 'Sweep_checkmark',
 'Static_fist',
 'Measure(distance)',
 'Photo_frame',
 'Number_0',
 'Number_1',
 'Number_2',
 'Number_3',
 'Number_4',
 'Number_5',
 'Number_6',
 'Number_7',
 'Number_8',
 'Number_9',
 'OK',
 'Another_number_3',
'Pause',
 'Shape_C',
 'Make_a_phone_call',
'Wave_hand',
 'Wave_finger',
 'Knock',
 'Beckon',
 'Palm_to_fist',
 'Fist_to_Palm',
 'Trigger_with_thumb',
 'Trigger_with_index_finger',
 'Hold_fist_in_the_other_hand',
 'Grasp',
 'Walk',
 'Gather_fingers',
 'Snap_fingers',
 'Applaud',
 'Dual_hands_heart',
 'Put_two_fingers_together',
 'Take_two_fingers_apart',
 'Turn_over',
 'Move_fist_upward',
 'Move_fist_downward',
 'Move_fist_toward_left',
 'Move_fist_toward_right',
 'Bring_hand_close',
 'Push_away',
 'Thumb_upward',
 'Thumb_downward',
'Thumb_toward_right',
 'Thumb_toward_left',
 'Thumbs_backward',
 'Thumbs_forward',
 'Move_hand_upward',
 'Move_hand_downward',
 'Move_hand_towards_left',
 'Move_hand_towards_right',
 'Draw_circle_with_hand_in_horizontal_surface',
'Bent_number_2',
'Bent_another_number_3',
'Dual_fingers_heart',
'Scroll_fingers_toward_left',
'Scroll_fingers_toward_right',
'Move_fingers_upward',
'Move_fingers_downward',
'81 Move_fingers_left',
'Move_fingers_right',
'Move_fingers_forward']

In [9]:
# Resize frames
import matplotlib.image as img

def resize_frame(frame, size = (64,64)):
    frame = img.imread(frame)
    frame = cv2.resize(frame, size)
    return frame

In [10]:
# return gray image
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [11]:
def show(img, title="img"):
    cv2.imshow(title, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [12]:
import time

In [13]:
import random
import os

def picker(sample_count):
    L = []
    for cl in sample_count:
        L = L + [(f'{cl[1]}')]*cl[0]
    random.shuffle(L)
    return L

def pick_sample(shuffled_list):
    a = random.choice(shuffled_list)
    return a

def VideoDataGenerator(data_path, frame_dim, batch_size = 8):
    '''The data_path must contain folders containing samples from only that class.
    The samples should be homogenously numerated. (eg - 001.jpg, 002.jpg etc) for each frame of the video. 
    Enter frame_dim as tuple of dimensions in the format: (x-axis, y-axis)
    Usage:
        data_path: string format path to the directory containing your classes
        frame_dim: a tuple of your width and height - (width, height) in pixels
        batch_size: number of batches to be produced by the yield function, recommended 1,2,4,8 for regular computers
    '''
    def resize_frame(frame, size = frame_dim):
        frame = img.imread(frame)
        frame = cv2.resize(frame, size)
        return frame
    if data_path[-1] != '/':
        data_path = f"{data_path}/"
    
    classes = os.listdir(data_path)
    print(f"{len(classes)} classes found in folder")
    
    class_target_map = [[classes[i], i] for i in range(len(classes))]
    samples_count = [[len(os.listdir(f"{data_path}{classes[i]}")), i] for i in range(len(classes))]
    picker_list = picker(samples_count)
    
    sample_names = [[] for i in range(len(classes))]
    for i in range(len(classes)):
        sample_names[i] = os.listdir(f'{data_path}{classes[i]}')
    
    while True:
        samples_pushed = []
        offset = 0
        total_samples = 0
        c = 0
        for cl in sample_names:
            if not cl:
                c += 1
            if c == len(classes) - 1:
                break
        X = []
        y = []
        for cl in classes:
            total_samples = total_samples + len(os.listdir(f'{data_path}{cl}'))
            
        for batch_iter in range(batch_size):
            sample_class = pick_sample(picker_list)
            picker_list.remove(sample_class)
            sample_class = int(sample_class)
            if not sample_names[sample_class]: 
                batch_iter -= 1
                continue
            vid_sample = f'{data_path}{classes[sample_class]}/{sample_names[sample_class][0]}'
            #fuck up hoga toh ye^ line me hoga
            frames = os.listdir(vid_sample)
            frames.sort()
            vid = []
            for frame in frames:
                frame = resize_frame(f'{vid_sample}/{frame}')
                try:
                    #frame = resize_frame(frame)
                    vid.append(frame)
                except:
                    print(f'Sample with tag - {sample_names[sample_class][0]} is broken and skipped.')
                    continue
            #print(np.shape(vid))
            #vid = np.reshape(vid, (96, 64, 32, 3))
            X.append(vid)
            y.append(sample_class)
            
            sample_names[sample_class].pop(0)
        X = np.array(X)
        y = np.array(y)
        yield X,y

In [14]:
training_path = '/home/siddhant/Datasets/EgoGesture/EgoGesture2'
validation_path = '/home/siddhant/Datasets/EgoGesture/test'

batchsize = 2
training_set = VideoDataGenerator(training_path, frame_dim=(96,64), batch_size=batchsize)
validation_set = VideoDataGenerator(validation_path, frame_dim=(96,64), batch_size=batchsize)

In [15]:
sgd = SGD(lr=0.001)
model.compile(optimizer=sgd, loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
inst = next(training_set)

83 classes found in folder


In [17]:
np.shape(inst[0])

(2, 16, 64, 96, 3)

In [18]:
import sys
sys.getsizeof(inst[0])

589984

In [19]:
checkpoints = ModelCheckpoint(
    os.getcwd(),
    monitor="val_loss",
    verbose=0,
    save_best_only=False, 
    save_weights_only=False,
    mode="auto",
    save_freq="epoch"
)


In [20]:
#gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)

In [21]:
model.load_weights('EGO.h5')

In [44]:
training_history = model.fit_generator(training_set, use_multiprocessing=True,
                                       epochs =1,validation_data = validation_set , verbose = 1 , validation_steps = 1,
                                       steps_per_epoch=140000//16, callbacks = [checkpoints])

8749/8750 [============================>.] - ETA: 0s - loss: 0.2494 - accuracy: 0.932083 classes found in folder
INFO:tensorflow:Assets written to: /home/siddhant/Codes/3D EgoGesture/assets
8750/8750 [==============================] - 3041s 348ms/step - loss: 0.2494 - accuracy: 0.9321 - val_loss: 0.0297 - val_accuracy: 1.0000


In [48]:
model.save_weights(filepath='EGO.h5')

In [49]:
model.load_weights('EGO.h5')

In [29]:
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
cap.set(12, 50)
cap.set(6, 10)
classe = ''
import time 
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_cp = frame
    #print(cap.get(6))
    frame_cp = cv2.resize(frame, (96, 64))
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    to_predict.append(frame_cp)
    
    if len(to_predict) == 16:
        print(".", end="")
        frame_to_predict = [[]]
        frame_to_predict[0] = np.array(to_predict, dtype=np.float32)
        #frame_to_predict = normaliz_data(frame_to_predict)
        #print(frame_to_predict)
        predict = model.predict(np.array(frame_to_predict))
        time.sleep(0.1)
        classe = classes[np.argmax(predict)]
        if np.argmax(predict)!=2:
            print('Class = ',classe, 'Precision = ', np.amax(predict)*100,'%')


        #print(frame_to_predict)
        to_predict = []
        time.sleep(0.1) # Time in seconds
        font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, classe, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0.5, 0.5),1,cv2.LINE_AA)


    # Display the resulting frame
    cv2.imshow('Hand Gesture Recognition',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

.Class =  Measure(distance) Precision =  11.85995489358902 %
.Class =  Knock Precision =  11.954901367425919 %
.Class =  Knock Precision =  12.476347386837006 %
.Class =  Knock Precision =  8.666125684976578 %
.Class =  Knock Precision =  8.861304074525833 %
.Class =  Knock Precision =  8.924122154712677 %
.Class =  Knock Precision =  9.842980653047562 %
.Class =  Knock Precision =  8.927010744810104 %
.Class =  Knock Precision =  9.655726701021194 %
.Class =  Knock Precision =  9.927326440811157 %
.Class =  Knock Precision =  10.277998447418213 %
.Class =  Knock Precision =  9.683194011449814 %
.Class =  Knock Precision =  10.13307198882103 %
.Class =  Turn_over Precision =  57.19076991081238 %
.Class =  Bent_another_number_3 Precision =  8.53370726108551 %
.Class =  Bent_another_number_3 Precision =  8.971205353736877 %
.Class =  Put_two_fingers_together Precision =  44.46413815021515 %
.Class =  Bent_another_number_3 Precision =  21.286572515964508 %
.Class =  Number_8 Precision =  

In [32]:
testing_set = VideoDataGenerator(validation_path, frame_dim=(96,64), batch_size=16)
inst = next(testing_set)

score = model.evaluate(inst[0], inst[1], verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

83 classes found in folder
Test loss: 0.32248181104660034
Test accuracy: 0.875
